In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import numpy as np

torch.manual_seed(2019)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [4]:
train_data = torchvision.datasets.CIFAR100('./data/', train=True, transform= trans, download=True)
test_data = torchvision.datasets.CIFAR100('./data/', train=False, transform= trans, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
batch_size = 64

conv_channel_1 = 32
conv_channel_2 = 32
conv_channel_3 = 64
conv_channel_4 = 64
conv_channel_5 = 128

hidden1_size = 1024
#hidden2_size = 300
hidden2_size = 1024
out_size = 100
epoch = 50

In [0]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,shuffle=True)

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

import time

In [0]:
loss = nn.CrossEntropyLoss()

In [0]:
import torch.optim as optim
from torch.autograd import Variable

def fit(epoch, data_loader, model, opt, phase):
    if phase=='train':
        volatile = False
        model.train()
    else:
        volatile = True
        model.eval()       
        
    learning_loss = 0.
    learning_correct = 0
    
    for i, (inputs, labels) in enumerate(data_loader):
        if phase == 'train':
            opt.zero_grad()
            
        inputs = Variable(inputs, volatile).cuda()
        labels = Variable(labels).cuda()
        outputs = model(inputs)
        
        loss_value = loss(outputs, labels)
        learning_loss += loss_value.item() * inputs.shape[0]
        
        if phase == 'train':
            loss_value.backward()
            opt.step()
        
        _, preds = torch.max(outputs.data, 1)
        learning_correct += (preds == labels).sum().item()
        
    epoch_loss = learning_loss /len(data_loader.dataset)
    epoch_correct = learning_correct / len(data_loader.dataset) * 100.
    
    print(f'epoch = {epoch} -> {phase:{5}} / loss = {epoch_loss:{8}.{3}}, correct = {epoch_correct:{8}.{4}}%')

In [0]:
class commonLinear(nn.Module):
    def __init__(self, nin):
        super(commonLinear, self).__init__()
        self.fc=nn.Sequential(
            nn.Linear(nin, hidden2_size),
            nn.ReLU(),
#             nn.Linear(hidden1_size, hidden2_size),
#             nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden2_size, out_size),
            nn.ReLU()
        )
        
    def forward(self,x):
        x=self.fc(x)
        return F.log_softmax(x, dim=1)

In [0]:
class normalCNN(nn.Module):
    def __init__(self):
        super(normalCNN,self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, conv_channel_1, 3, padding=1),
            nn.BatchNorm2d(conv_channel_1),
            nn.ReLU(),
            nn.Conv2d(conv_channel_1, conv_channel_2, 3, padding=1),
            nn.BatchNorm2d(conv_channel_2),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(conv_channel_2, conv_channel_3, 3, padding=1),
            nn.BatchNorm2d(conv_channel_3),
            nn.ReLU(),
            nn.Conv2d(conv_channel_3, conv_channel_4, 3, padding=1),
            nn.BatchNorm2d(conv_channel_4),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(conv_channel_4, conv_channel_5, 3, padding=1),
            nn.BatchNorm2d(conv_channel_5),
            nn.ReLU(),
            nn.AvgPool2d(2,2)
        )
        
        self.fc = commonLinear(conv_channel_5*4*4)
        
    def forward(self,x):
        x=self.conv(x)
        x=x.view(-1,conv_channel_5*4*4)
        x=self.fc(x)
        return x

In [12]:
cnn1 = normalCNN().cuda()
opt1 = optim.Adadelta(cnn1.parameters(), rho=0.95)
#opt1 = optim.SGD(cnn1.parameters(), lr=0.01, momentum=0.9)
print(cnn1)

normalCNN(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [13]:
for ep in range(epoch):
    t= time.time()
    fit(ep, train_loader, cnn1, opt1, 'train')
    fit(ep, test_loader, cnn1, opt1, 'eval')
    print('%.4f'%(time.time()-t))
    print()

epoch = 0 -> train / loss =     4.54, correct =    3.494%
epoch = 0 -> eval  / loss =     4.42, correct =     6.84%
37.0818

epoch = 1 -> train / loss =     4.38, correct =    7.738%
epoch = 1 -> eval  / loss =     4.27, correct =    10.06%
37.1036

epoch = 2 -> train / loss =     4.24, correct =    10.81%
epoch = 2 -> eval  / loss =     4.18, correct =    12.61%
37.0091

epoch = 3 -> train / loss =     4.13, correct =    13.59%
epoch = 3 -> eval  / loss =     4.09, correct =    14.83%
37.0811

epoch = 4 -> train / loss =     4.06, correct =    15.29%
epoch = 4 -> eval  / loss =     4.04, correct =    16.34%
37.0371

epoch = 5 -> train / loss =     3.96, correct =    17.65%
epoch = 5 -> eval  / loss =     3.93, correct =    18.47%
37.1221

epoch = 6 -> train / loss =     3.84, correct =    20.54%
epoch = 6 -> eval  / loss =     3.79, correct =    21.37%
37.0679

epoch = 7 -> train / loss =      3.7, correct =    23.48%
epoch = 7 -> eval  / loss =     3.72, correct =    23.41%
36.9754



In [0]:
class depthwiseCNN(nn.Module):
    def __init__(self,nin,nout):
        super(depthwiseCNN,self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(nin,nin,3,groups=nin, padding=1),
            nn.BatchNorm2d(nin),
            nn.ReLU(),
            nn.Conv2d(nin,nout,1),
            nn.BatchNorm2d(nout),
            nn.ReLU()
        )

    def forward(self,x):
        return self.model(x)
        
class separableCNN(nn.Module):
    def __init__(self):
        super(separableCNN,self).__init__()
        self.conv = nn.Sequential(
            depthwiseCNN(3,conv_channel_1),
            depthwiseCNN(conv_channel_1,conv_channel_2),
            nn.MaxPool2d(2,2),
            depthwiseCNN(conv_channel_2,conv_channel_3),
            depthwiseCNN(conv_channel_3,conv_channel_4),
            nn.MaxPool2d(2,2),
            depthwiseCNN(conv_channel_4,conv_channel_5),
            nn.AvgPool2d(2,2)
        )
        
        self.fc = commonLinear(conv_channel_5*4*4)
        
    def forward(self,x):
        x=self.conv(x)
        x=x.view(-1,conv_channel_5*4*4)
        x=self.fc(x)
        return x

In [17]:
cnn2 = separableCNN().cuda()
opt2 = optim.Adadelta(cnn2.parameters(), rho=0.95)
#opt2 = optim.SGD(cnn1.parameters(), lr=0.01, momentum=0.9)
print(cnn2)

separableCNN(
  (conv): Sequential(
    (0): depthwiseCNN(
      (model): Sequential(
        (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=3)
        (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
        (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
      )
    )
    (1): depthwiseCNN(
      (model): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
        (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
      )
    )
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

In [0]:
for ep in range(epoch):
    t=time.time()
    fit(ep, train_loader, cnn2, opt2, 'train')
    fit(ep, test_loader, cnn2, opt2, 'eval')
    print('%.4f'%(time.time()-t))
    print()


epoch = 0 -> train / loss =      4.6, correct =    1.428%
epoch = 0 -> eval  / loss =     4.57, correct =     2.95%
32.5999

epoch = 1 -> train / loss =     4.49, correct =      4.9%
epoch = 1 -> eval  / loss =      4.4, correct =     7.52%
33.6216

epoch = 2 -> train / loss =      4.3, correct =    9.488%
epoch = 2 -> eval  / loss =     4.21, correct =    12.22%
32.5093

epoch = 3 -> train / loss =     4.15, correct =    13.07%
epoch = 3 -> eval  / loss =     4.27, correct =    10.29%
33.7230

epoch = 4 -> train / loss =     4.07, correct =    14.93%
epoch = 4 -> eval  / loss =     4.09, correct =    14.84%
32.6653

epoch = 5 -> train / loss =     3.98, correct =     16.8%
epoch = 5 -> eval  / loss =     3.95, correct =    17.38%
33.6847

epoch = 6 -> train / loss =     3.86, correct =    20.11%
epoch = 6 -> eval  / loss =     3.84, correct =    20.53%
32.7005

epoch = 7 -> train / loss =     3.75, correct =    22.36%
epoch = 7 -> eval  / loss =     3.77, correct =    22.02%
33.2745



In [0]:
class inception(nn.Module):
    def __init__(self, nin, nout):
        super(inception,self).__init__()
        self.model1x1 = nn.Sequential(
            nn.Conv2d(nin, int(nout/4), 1)
        )
        self.model3x3 = nn.Sequential(
            nn.Conv2d(nin,int(nout/8),1),
            nn.Conv2d(int(nout/8),int(nout/4),3, padding=1)
        )
        self.model5x5 = nn.Sequential(
            nn.Conv2d(nin,int(nout/16),1),
            nn.Conv2d(int(nout/16),int(nout/8),3,padding=1),
            nn.Conv2d(int(nout/8),int(nout/4),3,padding=1)
        )
        self.modelmax = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(nin,int(nout/4),1)
        )
        
    def forward(self, x):
        part1x1=self.model1x1(x)
        part3x3=self.model3x3(x)
        part5x5=self.model5x5(x)
        partmax=self.modelmax(x)
        outputs= [part1x1, part3x3, part5x5, partmax]
        return torch.cat(outputs,1)
    
class inceptionCNN(nn.Module):
    def __init__(self):
        super(inceptionCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3,conv_channel_1, 3, padding=1),
            nn.BatchNorm2d(conv_channel_1),
            nn.ReLU(),
            nn.Conv2d(conv_channel_2, conv_channel_2, 3, padding=1),
            nn.BatchNorm2d(conv_channel_1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(conv_channel_1, conv_channel_2, 3, padding=1),
            nn.BatchNorm2d(conv_channel_2),
            nn.ReLU(),
            inception(conv_channel_2, conv_channel_3),
            nn.BatchNorm2d(conv_channel_3),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            inception(conv_channel_3, conv_channel_4),
            nn.BatchNorm2d(conv_channel_4),
            nn.ReLU(),
            inception(conv_channel_4, conv_channel_5),
            nn.BatchNorm2d(conv_channel_5),
            nn.ReLU(),
            nn.AvgPool2d(2,2)
        )
        
        self.fc=commonLinear(conv_channel_5*4*4)
    
    def forward(self,x):
        x=self.conv(x)
        x=x.view(-1, conv_channel_5*4*4)
        x=self.fc(x)
        return x

In [23]:
cnn3 = inceptionCNN().cuda()
opt3 = optim.Adadelta(cnn3.parameters(), rho=0.95)
#opt3 = optim.SGD(cnn3.parameters(), lr=0.01, momentum=0.9)
print(cnn3)

inceptionCNN(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): inception(
      (model1x1): Sequential(
        (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
      )
      (model3x3): Sequential(
        (0): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
        (1): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (model5x5): Sequential(
        (0)

In [24]:
for ep in range(epoch):
    t=time.time()
    fit(ep, train_loader, cnn3, opt3, 'train')
    fit(ep, test_loader, cnn3, opt3, 'eval')
    print('%.4f'%(time.time()-t))
    print()


epoch = 0 -> train / loss =     4.38, correct =     5.56%
epoch = 0 -> eval  / loss =     4.03, correct =    11.82%
30.8125

epoch = 1 -> train / loss =     3.75, correct =    13.58%
epoch = 1 -> eval  / loss =     3.46, correct =    18.35%
30.7551

epoch = 2 -> train / loss =     3.34, correct =     19.7%
epoch = 2 -> eval  / loss =     3.16, correct =    24.37%
30.5909

epoch = 3 -> train / loss =     3.01, correct =     25.8%
epoch = 3 -> eval  / loss =      2.9, correct =    28.41%
30.5110

epoch = 4 -> train / loss =     2.76, correct =    30.55%
epoch = 4 -> eval  / loss =     2.78, correct =    31.43%
30.6968

epoch = 5 -> train / loss =     2.57, correct =     34.3%
epoch = 5 -> eval  / loss =     2.64, correct =    33.51%
30.5297

epoch = 6 -> train / loss =      2.4, correct =    37.28%
epoch = 6 -> eval  / loss =     2.33, correct =    39.76%
30.5465

epoch = 7 -> train / loss =     2.26, correct =    40.15%
epoch = 7 -> eval  / loss =     2.31, correct =    39.36%
30.7576

